In [ ]:
## Optimization of 2d Truss structures
import numpy as np
import csv
import pandas as pd

#from google.colab import drive
#drive.mount('/content/drive/')

(This list is subject to a lot of change. There will be more questions that will need an answer as we get further along, but this should suffice for now to give a GREAT starting point.) I will save as a revision so that I can track my own process)
What are all of our current questions/ problems?
How to import the data?
How to correcctly sdtructure the data in terms of trying to keep in mind the global matrix?
How to use the 4 given files and its data to:
  create the flexibility matrix
  create the forces on the nodes in 2D vector
  create the internal forces of each member vector
  have the code decide what we do and do not know  (think about A_f, A_p, F_f, and F_p)
  
  `


In [ ]:
'''
Helper functions
'''
'''
Build matrix to concatonate matrix to the correct size
'''
# COME BACK TO THIS
def build_matrix(rows, cols, var): # var is the variable that fills the matrix to build F, u, 
    matrix = []
  
    for r in range(0, rows): #remember this needs to call the 
      
      matrix.append([var for c in range(0, cols)])
  
    return matrix



'''
Calculate the distance between two points
'''
def dist(n0,n1):
    return ((n1[0]-n0[0])**2+(n1[1]-n0[1])**2)**0.5

'''
function
   plotTruss - this function plots the optimized truss
   to test that you can read the input correctly, you can start by setting
   elem['a'] = np.ones(num['m']) to test
input:
   glb - dictionary
   nodes - list of nodes as read from file [nx3]
   n0 - list of indices of the node at the start of the member [mx1]
   n1 - list of indices of the node at the end of the member [mx1]
   a - optimized cross sectional area as an array [mx1]
'''
def plotTruss(glb, nodes, n0, n1, a):
    
    from mpl_toolkits import mplot3d
    import matplotlib.pyplot as plt
    
    fig = plt.figure(figsize=(12, 12))
    ax = plt.axes()
    
    # draw line by line, assigning each a line width equalling the diameter
    for i, [n0i,n1i] in enumerate(zip(n0,n1)):
        x = [[nodes[n0i][i], nodes[n1i][i]] for i in [0,1]]
        ax.plot(x[0],x[1], c='b',linewidth=2*np.sqrt(a[i]/np.pi)*500)

    plt.show()
    

In [ ]:
import pandas as pd
'''
from google.colab import files
print('Please upload constraints.csv, forces.csv, members.csv, nodes.csv ')
uploaded = files.upload() # four test files have been loaded now need to create the dictionary with these files
'''

cf= pd.read_csv("constraints.csv", header=None)
constraints_arr =cf.to_numpy()
c = np.shape(constraints_arr)[0]

ff = pd.read_csv("forces.csv", header=None)
forces_arr =ff.to_numpy()
f = np.shape(forces_arr)[0]

nf= pd.read_csv("nodes.csv", header=None)
nodes_arr =nf.to_numpy()
n = np.shape(nodes_arr)[0]  # row size of the A matrix

mf= pd.read_csv("members.csv", header=None)
members_arr = mf.to_numpy()
m = np.shape(members_arr)[0] #columns size of the A matrix

print('\n')
#print(inputData)
print("\nThis is the dict for the nodess \n")
inputData = {'Nodes': nodes_arr , 'Members': members_arr, 'Constraints': constraints_arr, 'Forces': forces_arr} # Finish creating this dictionary with the other 3 sets of data
# The input dictionary has been create, now need to create the num dictionary
print(inputData)
 
num = {'c': c, 'f': f, 'n': n, 'm': m, 'nDOF': 2, 'DOF': 2*n}
print(num)




This is the dict for the nodess 

{'Nodes': array([[0.        , 0.        ],
       [2.        , 0.        ],
       [4.        , 0.        ],
       [1.        , 1.73205081],
       [3.        , 1.73205081]]), 'Members': array([[0, 1],
       [0, 3],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 4],
       [3, 4]]), 'Constraints': array([[0, 0, 1, 0],
       [2, 1, 1, 0]]), 'Forces': array([[     1,      0,      1, -50000]])}
{'c': 2, 'f': 1, 'n': 5, 'm': 7, 'nDOF': 2, 'DOF': 10}


In [ ]:


cf= pd.read_csv("constraints.csv", header=None)
constraints_arr =cf.to_numpy()
c = np.shape(constraints_arr)[0]

ff = pd.read_csv("forces.csv", header=None)
forces_arr =ff.to_numpy()
f = np.shape(forces_arr)[0]

nf= pd.read_csv("nodes.csv", header=None)
nodes_arr =nf.to_numpy()
n = np.shape(nodes_arr)[0]  # row size of the A matrix

mf= pd.read_csv("members.csv", header=None)
members_arr = mf.to_numpy()
m = np.shape(members_arr)[0] #columns size of the A matrix

print('\n')
#print(inputData)
print("\nThis is the dict for the nodess \n")
inputData = {'Nodes': nodes_arr , 'Members': members_arr, 'Constraints': constraints_arr, 'Forces': forces_arr} # Finish creating this dictionary with the other 3 sets of data
# The input dictionary has been create, now need to create the num dictionary
print(inputData)
 
num = {'c': c, 'f': f, 'n': n, 'm': m, 'nDOF': 2, 'DOF': 2*n}
print(num)





This is the dict for the nodess 

{'Nodes': array([[0.        , 0.        ],
       [2.        , 0.        ],
       [4.        , 0.        ],
       [1.        , 1.73205081],
       [3.        , 1.73205081]]), 'Members': array([[0, 1],
       [0, 3],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 4],
       [3, 4]]), 'Constraints': array([[0, 0, 1, 0],
       [2, 1, 1, 0]]), 'Forces': array([[     1,      0,      1, -50000]])}
{'c': 2, 'f': 1, 'n': 5, 'm': 7, 'nDOF': 2, 'DOF': 10}


### Function that takes in a directory and imports all the input files (You should code this)

In [ ]:
'''
function: 
   Read data from file
input:
   cwd - path to the input files
output: # At somepoint we SHOULD have the code verify the data inside the .csv because if there is string and numbers in the same file 
        # it can complicate and cause the code to have an error, or continuously loop
   inputData - dictionary containing
       inputData['constraints'] something like {'constraints' : somethingsomething import 7_bar/constraints.csv }
       inputData['forces'] something like {'forces' : somethingsomething import 7_bar/forces.csv }
       inputData['members']   {'members': import 7_bar/members}
       inputData['nodes'] 
   num - dictionary containing
       num['c'] - number of constraints
       num['f'] - number of forces
       num['m'] - number of members # Needs to be able to extract the length or size from the corresponding file or matrix
       num['n'] - number of nodes # Needs to be able to extract the length or size from the corresponding file or matrix
       num['nDOF'] - number of degreees of freedom per node = 2
       num['DOF'] - total number of degrees of freedom = 2n
'''
# Need to make sure that the program is pulling the correct data files right now 
# it is pulling the 7 bar files that have been uploaded to the program

def importData(cwd): 
    
   cf= pd.read_csv("constraints.csv", header=None) #pandas was taking the first row and thinking it was a header
   constraints_arr =cf.to_numpy()
   c = np.shape(constraints_arr)[0]

   ff = pd.read_csv("forces.csv", header=None)
   forces_arr =ff.to_numpy()
   f = np.shape(forces_arr)[0]

   nf= pd.read_csv("nodes.csv", header=None)
   nodes_arr =nf.to_numpy()
   n = np.shape(nodes_arr)[0]  # row size of the A matrix

   mf= pd.read_csv("members.csv", header=None)
   members_arr = mf.to_numpy()
   m = np.shape(members_arr)[0] #columns size of the A matrix


   inputData = {'Nodes': nodes_arr , 'Members': members_arr, 'Constraints': constraints_arr, 'Forces': forces_arr} # Finish creating this dictionary with the other 3 sets of data
   # The input dictionary has been create, now need to create the num dictionary

   num = {'c': c, 'f': f, 'n': n, 'm': m, 'nDOF': 2, 'DOF': 2*n}

    
   return [inputData, num]  

In [ ]:
force_vec = np.zeros((2*num['n'],1))
# Global Force Vector
# For Systems with one force

print(inputData['Forces'][1)
'''
if num['f'] ==1 :
  if inputData['Forces'][1] == 1:
      force_vec[int(2*inputData['Forces'][0]),0] = inputData['Forces'][3]
  elif inputData['Forces'][2]==1:
      force_vec[int(inputData['Forces'][row,0]+1),0] == inputData['Force'][3]
print('This is Force vector 1')
print(force_vec)

# For Systems with more than one force
    if num['f'] >1:
      for row in np.arange(0,num['f']):
        if inputData['Forces'][row,1] == 1:
          force_vec[int(2*inputData['Forces'][row,1]),0] = inputData['Forces'][row,3]
      elif inputData['Forces'][row,2] == 1:
        force_vec[int(2*inputData['Forces'][row, 0]+1),0] = inputData['Forces'][row,3]

# Global Displacement Vector
    displacement_vec = np.ones((2*num['n'],1))*np.inf
    for row in np.arange(0,num['c']):
      if inputData['Constraints'][row,1] == 1: 
        displacement_vec[int(2*inputData['Constraints'][row,0]),0] = inputData['Constraints'][row,3]
      if inputData['Constraints'][row,0]+1),0] = inputData['Constraints'][row,3]


    #freedom degrees
  inf_loc = np.where(displacement_vec == np.inf)[0]
'''

IndexError: ignored

### Function that initializes the dictinoaries with the necessary ingredients (You should code this)

In [ ]:
'''
function:
   Initialization of material and geometrical properties
input:
   inputData
   num
output:
   glb - dictionary containing DOF related quantities
       glb['F'] - initial F array [2n x 1] - 0 excpet for the force conditions in the forces.csv file
       glb['u'] - initial u array [2n x 1] - infinite except the boundary conditions in the constraints.csv file
       glb['fDOF'] - row id# of all the free degree of freedom
   elem - dictionary containing member related quantities
       elem['L'] - array of element lengths [mx1]
       elem['n0'] - list of indices of the node at the start of the member [mx1]
       elem['n1'] - list of indices of the node at the end of the member [mx1]
'''
def initializeStruct(inputData,num):
    # if we are importing the other dictionaries then I should find a way to begin to make the new diction entries
   

    '''
        Type your code here
    '''
    
    return [glb, elem]    

### Function that constructs the adjacency matrix (You should code this)

In [ ]:

'''
construct the adjacency matrix
input:
       num
       nodes - list of nodes as read from file
       n0 - list of indices of the node at the start of the member
       n1 - list of indices of the node at the end of the member
output:
       A - adjacency matrix [2*n by m]
'''
def assembleA(num, nodes, n0, n1):
    
    '''
        Type your code here
    '''
    
    return A

### Function that optimizes the truss (You should code this)

In [ ]:

'''
function
   optTruss - this function should define c, A_eq, b_eq, A_ub, b_ub, bounds, and use 
   https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
input:
   glb
   elem
   num
   yieldStress
output:
   a_i - optimized cross sectional area as an array [mx1]
   a_i - element-wise forces [mx1]
   F   - global forces (inluding the reaction forces) [2nx1]
'''
def optTruss(glb, elem, num, yieldStress):
    from scipy.optimize import linprog


    '''
        Type your code here
    '''
    
    return [a_i,f_i, F]

### This is where all the functions you coded above is executed

In [ ]:

# Folder containing all your models
inputFolder='input/'

# Sub folder containing the particular model for analysis
inputModel='7_bar/'

#current working directory
cwd=inputFolder+inputModel

'''
function: 
   Read data from file
'''
[inputData, num] = importData(cwd)

'''
function:
   Initialization of material and geometrical properties
'''
[glb, elem] = initializeStruct(inputData, num)

'''
construct the adjacency matrix
Directly put the output A (adjacency matrix) into the dictionary glb['A']
'''
glb['A'] = assembleA(num, inputData['nodes'], elem['n0'], elem['n1'])

### The next block does the optimization
I broke it up from the previous just so you can check all the previous functions work before going here

In [ ]:
yieldStress = 70e6 #Pa - yield stress - do not change this

'''
function
   optTruss - this function should define c, A_eq, b_eq, A_ub, b_ub, bounds, and use 
   https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html
   
   Directly put the output [a_i,f_i, F] into the dictionary elem['a'],elem['f'], glb['F']
'''
[elem['a'],elem['f'], glb['F']] = optTruss(glb, elem, num,yieldStress)

In [ ]:
'''
function
   plotTruss - this function plots the optimized truss
'''
plotTruss(glb, elem, inputData, elem['a'])